# Base Pytorch Model Implementation

#### More Tests

In [83]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchattacks import PGD, FGSM, VANILA, GN, Jitter #Ben: trying out some more attacks
from torchvision import datasets, transforms
import torchvision.models as models
from torch.utils.data import DataLoader
import random

In [84]:
# temporary: copying the code to load the model
device = torch.device("cpu")

train_transform =  transforms.Compose([
    transforms.RandomResizedCrop(size = 224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#Test does not use augmentation, only normalization is being performed.
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(size = 224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Set the directory for saving the dataset
# data_dir = '/content/drive/My Drive/FinalProject1'
data_dir = './data'

# Load datasets, first time you need to download it (may take a while). After that it should just pull the local copy
train_dataset = datasets.Food101(root=data_dir, split='train', download=True, transform=train_transform)
test_dataset = datasets.Food101(root=data_dir, split='test', download=True, transform=test_transform)

# Dataloaders, may need to change # of workers or batchsize to improve performance
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True,num_workers=4)

# Pretrained model, efficient architecture
model = models.efficientnet_b2(weights='DEFAULT')

# If false, then pretrained weights do not change
# for val in model.features.parameters():
#   val.requires_grad = False

# Add one more layer to base model and then add an output layer
model.classifier = nn.Sequential(
    nn.Linear(model.classifier[1].in_features, 1024),
    nn.ReLU(),
    nn.Linear(1024, len(train_dataset.classes))
)

print("Loading model")
# Path to the .pth file
model_path = 'results/bestmodel.pth'

# Load the state dictionary from the .pth file
state_dict = torch.load(model_path)

# Load the state dictionary into the model
model.load_state_dict(state_dict)

# Put model onto gpu
model = model.to(device)


Loading model


In [85]:
def general_PGD(model, loss_function, data, proj_norm=2, radius=0.03, lr=0.01, steps=10):

  features, labels = data
  features = features.clone().detach().to(device)
  labels = labels.clone().detach().to(device)

  adv_features = features.clone().detach()

  #maximize loss wrt feature perturbations, for fixed network parameters
  for _ in range(steps):
    adv_features.requires_grad = True

    #model prediction
    pred = model(adv_features)

    #error calculation
    error = loss_function(pred, labels)

    #gradient descend
    grad = torch.autograd.grad(error, adv_features)[0] #grad:(1, 64, 3, 224, 224), where the first coordinate if the batch number?
    grad_norm = torch.norm(grad, p=proj_norm, dim=[1,2,3]) #normalize the gradient according to paper https://arxiv.org/pdf/1706.06083
    grad = grad / grad_norm.view(-1,1,1,1)

    adv_features = adv_features.detach() + lr * grad

    #projection: |features - adv_features|_{norm} < radius
    orig_diff = features - adv_features
    orig_diff_norm = torch.norm(orig_diff, p=proj_norm, dim=[1,2,3])
    normalization = radius / orig_diff_norm

    diff = orig_diff * normalization.view(-1,1,1,1)

    adv_features = (features - diff).detach()

  return adv_features

In [94]:
# parameters setting
loss_function = nn.CrossEntropyLoss()
proj_norm=2
lr=0.001
radius=0.01

num_batches = 10 # how many batches to use?
data = next(iter(test_loader))
for i in range(num_batches - 1):
    batch = next(iter(test_loader))
    data[0] = torch.concat((data[0], batch[0]))
    data[1] = torch.concat((data[1], batch[1]))

#### Now, we'll try some different attacks (including some random ones), and see which ones perform well

In [95]:
eps = 0.03 # the adversarial tolerance

# FGSM (Fast Gradient Sign Method) attack
fgsm_attack = FGSM(model, eps=eps)

# Gaussian noise attack
gaussian_noise_attack = GN(model, std=eps)

# Jitter attack
jitter_attack = Jitter(model, eps=eps, steps=10)


# Iterate through the test loader
images, labels = data

# adv_features = general_PGD(model, loss_function, data, proj_norm=2, radius=0.03, lr=0.01, steps=10)
pgd_attack = PGD(model, eps=eps)

adv_images_fgsm, adv_images_gaussian,  adv_images_jitter, adv_images_pgd, adv_images_nothing =\
 None, None, None, None, images


# Apply attacks
adv_images_fgsm = images.to(device) + fgsm_attack(images, labels)
adv_images_gaussian = images.to(device) + gaussian_noise_attack(images, labels)
adv_images_jitter = images.to(device) + jitter_attack(images, labels)
adv_images_pgd = images.to(device) + pgd_attack(images, labels)

#### Finally, we'll evaluate how accurately the model can predict different adversarial inputs.

In [96]:
import torch
from torch.utils.data import Dataset, DataLoader

# Define a custom dataset to hold the adversarial examples
class AdversarialDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [97]:
# Define a function to evaluate model accuracy
def evaluate_model(model, dataloader):
    device = next(model.parameters()).device
    correct = 0
    total = 0

    # Set model to evaluation mode
    model.eval()

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images).to(device)

            # Get predicted labels
            _, predicted = torch.max(outputs, 1)

            # Update total and correct predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            

    # Calculate accuracy
    accuracy = (correct / total) * 100
    print('Accuracy on adversarial examples: {:.2f}%'.format(accuracy))



In [98]:
attacks = ["none", "fgsm", "gaussian noise", "jitter", "pgd"]

for attack, adv_data in zip(attacks, [adv_images_nothing, adv_images_fgsm, adv_images_gaussian, adv_images_jitter, adv_images_pgd]):
  print(f"Adversarial Attack with attack={attack}")
  # Convert adversarial examples to a dataset
  adversarial_dataset = AdversarialDataset(adv_data, labels)

  # Define a DataLoader for the adversarial examples
  adversarial_loader = DataLoader(adversarial_dataset, batch_size=100, shuffle=False)
  evaluate_model(model, adversarial_loader)

Adversarial Attack with attack=none
Accuracy on adversarial examples: 77.81%
Adversarial Attack with attack=fgsm
Accuracy on adversarial examples: 41.56%
Adversarial Attack with attack=gaussian noise
Accuracy on adversarial examples: 67.19%
Adversarial Attack with attack=jitter
Accuracy on adversarial examples: 67.19%
Adversarial Attack with attack=pgd
Accuracy on adversarial examples: 62.50%
